## Sparse Identification of Nonlinear Dynamics (SINDy-PDE) for Burgers Eq.


In [1]:
!pip install numpy scipy scikit-learn matplotlib

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import Lasso
from scipy.io import loadmat
from scipy.interpolate import griddata

In [3]:
# --- Simulated data (replace with your real data) ---
# We'll generate data from the Burgers’ equation analytically

def burgers_solution(x, t, nu=0.1):
    # A rough solution using Cole-Hopf transformation (for demonstration)
    u = -2 * nu * np.pi * np.sin(np.pi * x) * np.exp(-np.pi**2 * nu * t)
    u /= (1 + np.cos(np.pi * x) * np.exp(-np.pi**2 * nu * t))
    return u

In [4]:
# Grid
nx, nt = 100, 100
x = np.linspace(-1, 1, nx)
t = np.linspace(0, 1, nt)
X, T = np.meshgrid(x, t)
U = burgers_solution(X, T, nu=0.1)

<ipython-input-3-925991f73ed1>:7: RuntimeWarning: divide by zero encountered in divide
  u /= (1 + np.cos(np.pi * x) * np.exp(-np.pi**2 * nu * t))


In [5]:
# --- Compute derivatives numerically ---
def finite_diff(U, dx, dt):
    u_x = (np.roll(U, -1, axis=1) - np.roll(U, 1, axis=1)) / (2 * dx)
    u_xx = (np.roll(U, -1, axis=1) - 2*U + np.roll(U, 1, axis=1)) / (dx**2)
    u_t = (np.roll(U, -1, axis=0) - np.roll(U, 1, axis=0)) / (2 * dt)
    return u_t[1:-1,1:-1], u_x[1:-1,1:-1], u_xx[1:-1,1:-1], U[1:-1,1:-1]


In [6]:
dx = x[1] - x[0]
dt = t[1] - t[0]
u_t, u_x, u_xx, u = finite_diff(U, dx, dt)

# Flatten everything for regression
u_t = u_t.flatten()
u_x = u_x.flatten()
u_xx = u_xx.flatten()
u = u.flatten()

# --- Construct the library of candidate terms ---
Theta = np.stack([
    u,                  # u
    u**2,               # u^2
    u_x,                # u_x
    u*u_x,              # u*u_x
    u_xx                # u_xx
], axis=1)

feature_names = ['u', 'u^2', 'u_x', 'u*u_x', 'u_xx']

In [7]:
# --- Perform sparse regression ---
lasso = Lasso(alpha=1e-4)  # Regularization strength
lasso.fit(Theta, u_t)
xi = lasso.coef_

In [8]:
# --- Display results ---
print("Identified PDE:")
print("u_t = ", end="")
for i, coef in enumerate(xi):
    if abs(coef) > 1e-6:
        print(f"{coef:+.5f} * {feature_names[i]} ", end="")
print()

Identified PDE:
u_t = +0.88330 * u +0.48450 * u*u_x +0.15404 * u_xx 


## Symbolic Regression for Burgers Eq.

In [9]:
pip install gplearn

In [10]:
import numpy as np
from gplearn.genetic import SymbolicRegressor
from gplearn.functions import make_function
import matplotlib.pyplot as plt

In [11]:
# --- Step 1: Generate synthetic data (e.g., from Burgers’ equation) ---
def burgers_u(x, t, nu=0.1):
    return -2 * nu * np.pi * np.sin(np.pi * x) * np.exp(-np.pi**2 * nu * t) / (
        1 + np.cos(np.pi * x) * np.exp(-np.pi**2 * nu * t))

In [12]:
nx, nt = 100, 50
x = np.linspace(-1, 1, nx)
t = np.linspace(0.01, 1, nt)
X, T = np.meshgrid(x, t)
U = burgers_u(X, T)

In [13]:
# --- Step 2: Compute numerical derivatives (central difference) ---
dx = x[1] - x[0]
dt = t[1] - t[0]

def central_diff(f, axis, dx):
    return (np.roll(f, -1, axis=axis) - np.roll(f, 1, axis=axis)) / (2 * dx)

In [14]:
u = U[1:-1, 1:-1]
u_t = central_diff(U, axis=0, dx=dt)[1:-1, 1:-1]
u_x = central_diff(U, axis=1, dx=dx)[1:-1, 1:-1]
u_xx = (np.roll(U, -1, axis=1) - 2*U + np.roll(U, 1, axis=1)) / dx**2
u_xx = u_xx[1:-1, 1:-1]

In [15]:
# --- Step 3: Prepare data for regression ---
X_data = np.stack([u.flatten(), u_x.flatten(), u_xx.flatten()], axis=1)
y_data = u_t.flatten()

In [16]:
# --- Step 4: Symbolic Regression using gplearn ---
sr = SymbolicRegressor(
    population_size=2000,
    generations=20,
    stopping_criteria=0.001,
    p_crossover=0.7,
    p_subtree_mutation=0.1,
    p_hoist_mutation=0.05,
    p_point_mutation=0.1,
    max_samples=0.9,
    verbose=1,
    parsimony_coefficient=0.01,
    random_state=42
)

sr.fit(X_data, y_data)


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:474: FutureWarning: `BaseEstimator._validate_data` is deprecated in 1.6 and will be removed in 1.7. Use `sklearn.utils.validation.validate_data` instead. This function becomes public and is part of the scikit-learn developer API.
  warnings.warn(


    |   Population Average    |             Best Individual              |
---- ------------------------- ------------------------------------------ ----------
 Gen   Length          Fitness   Length          Fitness      OOB Fitness  Time Left
   0    33.27      7.19054e+18        7         0.478529          0.43225      1.74m
   1     9.76      8.87052e+08       15         0.446925         0.478759      1.87m
   2     5.64          1344.16        7         0.429112         0.435625      1.55m
   3     5.15          1470.44        5         0.268462         0.374422      1.22m
   4     3.83      2.74214e+09        5         0.263678         0.417418     49.52s
   5     5.11          214.739        5         0.254842         0.496825     26.32s
   6     5.22      1.75641e+06        5         0.253872         0.505542     26.91s
   7     5.11           178.46        5         0.253412         0.509678     21.88s
   8     5.32          916.176        5         0.249803         0.542113  

SymbolicRegressor(max_samples=0.9, p_crossover=0.7, p_hoist_mutation=0.05,
                  p_point_mutation=0.1, p_subtree_mutation=0.1,
                  parsimony_coefficient=0.01, population_size=2000,
                  random_state=42, stopping_criteria=0.001, verbose=1)

In [17]:
# --- Step 5: Show result ---
print("\n📜 Discovered PDE term:")
print("u_t ≈", sr._program)


📜 Discovered PDE term:
u_t ≈ sub(mul(0.051, X2), X0)


In [18]:
# --- Optional: Evaluate performance ---
from sklearn.metrics import r2_score
y_pred = sr.predict(X_data)
print("R² score:", r2_score(y_data, y_pred))

R² score: 0.8493987591380956
